In [5]:
import tkinter as tk

class ChessGame:
    def __init__(self):
        self.board = self.initialize_board()
        self.current_turn = "white"

    def initialize_board(self):
        # Representing the board as a 2D list. Each piece is represented by a two-letter code.
        # The first letter is the color ('w' for white, 'b' for black) and the second letter is the type of the piece.
        # For example, "wp" is a white pawn, "bk" is a black king.
        return [
            ["br", "bn", "bb", "bq", "bk", "bb", "bn", "br"],
            ["bp", "bp", "bp", "bp", "bp", "bp", "bp", "bp"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["wp", "wp", "wp", "wp", "wp", "wp", "wp", "wp"],
            ["wr", "wn", "wb", "wq", "wk", "wb", "wn", "wr"]
        ]

    def print_board(self):
        for row in self.board:
            print(" ".join(row))
        print()
        
    def make_move(self, move):
        print(f"Received move: {move}")

        # Validate the move
        if not self.is_move_valid(move):
            print("Invalid move")
            return False

        # Parse move and get piece
        start, end = move.split()
        start_row, start_col = self.convert_to_row_col(start)
        end_row, end_col = self.convert_to_row_col(end)
        piece = self.board[start_row][start_col]

        # Check if it's the correct turn
        if not self.is_correct_turn(piece):
            print("It's not your turn")
            return False

        # Handle special moves
        if self.is_pawn_promotion(piece, end_row):
            piece = self.handle_pawn_promotion(piece, end)
        elif self.is_castling_move(start_row, start_col, end_row, end_col):
            return self.execute_castling_move(move)

        # Execute the move
        self.execute_move(start_row, start_col, end_row, end_col, piece)

        # Change turn
        self.change_turn()
        return True

    def is_move_valid(self, move):
        return move in self.get_possible_moves()

    def convert_to_row_col(self, notation):
        col, row = ord(notation[0]) - ord('a'), 8 - int(notation[1])
        return row, col

    def is_correct_turn(self, piece):
        return (piece.startswith(self.current_turn[0]))

    def is_pawn_promotion(self, piece, end_row):
        return piece[1] == 'p' and end_row in [0, 7]

    def handle_pawn_promotion(self, piece, end_notation):
        promotion_piece = end_notation[2] if len(end_notation) == 3 else 'q'
        print(f"Pawn promoted to {promotion_piece}")
        return piece[0] + promotion_piece

    def execute_move(self, start_row, start_col, end_row, end_col, piece):
        self.board[end_row][end_col] = piece
        self.board[start_row][start_col] = "--"
        print(f"Moved {piece} from {chr(start_col + ord('a'))}{8 - start_row} to {chr(end_col + ord('a'))}{8 - end_row}")

    def change_turn(self):
        self.current_turn = "black" if self.current_turn == "white" else "white"
        print(f"Turn changed to {self.current_turn}")

    def execute_castling_move(self, move):
        start, end = move.split()
        start_col, start_row = ord(start[0]) - ord('a'), 8 - int(start[1])
        end_col, end_row = ord(end[0]) - ord('a'), 8 - int(end[1])

        # Move the king
        self.board[end_row][end_col] = self.board[start_row][start_col]
        self.board[start_row][start_col] = "--"

        # Move the rook for king-side or queen-side castling
        if end_col == start_col + 2:  # King-side castling
            self.board[start_row][start_col + 1] = self.board[start_row][7]  # Move rook next to king
            self.board[start_row][7] = "--"
        elif end_col == start_col - 2:  # Queen-side castling
            self.board[start_row][start_col - 1] = self.board[start_row][0]  # Move rook next to king
            self.board[start_row][0] = "--"

        # Update flags to indicate the king and rook have moved
        self.set_king_moved(self.current_turn)
        self.set_rook_moved_for_castling(start_row, end_col)

    def is_valid_move(self, move):
        print(f"Debug: Checking validity of move: {move}")

        start, end = move.split()
        start_row, start_col = self.convert_to_row_col(start)
        end_row, end_col = self.convert_to_row_col(end)

        piece = self.board[start_row][start_col]
        target = self.board[end_row][end_col]

        print(f"Debug: Start Position - Row: {start_row}, Column: {start_col}, Piece: {piece}")
        print(f"Debug: End Position - Row: {end_row}, Column: {end_col}, Target: {target}")

        if piece == "--":
            print("Debug: No piece at start position.")
            return False

        if self.is_castling_move(start_row, start_col, end_row, end_col):
            return self.validate_castling_move(move)

        return self.is_piece_move_valid(piece, start_row, start_col, end_row, end_col, target)

    def convert_to_row_col(self, notation):
        col, row = ord(notation[0]) - ord('a'), 8 - int(notation[1])
        return row, col

    def is_piece_move_valid(self, piece, start_row, start_col, end_row, end_col, target):
        if piece[1] == "p":
            return self.is_valid_pawn_move(piece, start_row, start_col, end_row, end_col, target)
        elif piece[1] == "n":
            return self.is_valid_knight_move(piece, start_row, start_col, end_row, end_col, target)
        elif piece[1] == "r":
            return self.is_valid_rook_move(piece, start_row, start_col, end_row, end_col, target)
        elif piece[1] == "b":
            return self.is_valid_bishop_move(piece, start_row, start_col, end_row, end_col, target)
        elif piece[1] == "q":
            return self.is_valid_queen_move(piece, start_row, start_col, end_row, end_col, target)
        elif piece[1] == "k":
            return self.is_valid_king_move(piece, start_row, start_col, end_row, end_col, target)
        else:
            print("Debug: Unknown piece type.")
            return False
        
    def is_valid_pawn_move(self, piece, start_row, start_col, end_row, end_col, target):
        direction = 1 if piece[0] == 'w' else -1
        # Forward move
        if start_col == end_col:
            # Move one square forward
            if end_row == start_row - direction and target == "--":
                return True
            # Move two squares from initial position
            elif start_row == (6 if piece[0] == 'w' else 1) and end_row == start_row - 2 * direction:
                if target == "--" and self.board[start_row - direction][start_col] == "--":
                    return True
        # Diagonal capture
        elif abs(start_col - end_col) == 1 and end_row == start_row - direction:
            if target != "--" and target[0] != piece[0]:
                return True
        return False

    
    def is_valid_knight_move(self, piece, start_row, start_col, end_row, end_col, target):
        # Check if the target square is reachable by the knight
        if (abs(start_row - end_row) == 2 and abs(start_col - end_col) == 1) or \
           (abs(start_row - end_row) == 1 and abs(start_col - end_col) == 2):
            # Check if the target square is either empty or contains an opponent's piece
            return target == "--" or target[0] != piece[0]
        return False
    
    def is_valid_rook_move(self, piece, start_row, start_col, end_row, end_col, target):
        if start_row != end_row and start_col != end_col:
            return False  # Rook moves either horizontally or vertically

        step_row = 1 if end_row > start_row else -1 if end_row < start_row else 0
        step_col = 1 if end_col > start_col else -1 if end_col < start_col else 0

        row, col = start_row + step_row, start_col + step_col
        while row != end_row or col != end_col:
            if self.board[row][col] != "--":  # Check if path is blocked
                return False
            row, col = row + step_row, col + step_col

        return target == "--" or target[0] != piece[0]
    
    def is_valid_bishop_move(self, piece, start_row, start_col, end_row, end_col, target):
        # Bishop moves diagonally
        if abs(start_row - end_row) != abs(start_col - end_col):
            return False

        step_row = 1 if end_row > start_row else -1
        step_col = 1 if end_col > start_col else -1

        row, col = start_row + step_row, start_col + step_col
        while row != end_row and col != end_col:
            if self.board[row][col] != "--":  # Check if path is blocked
                return False
            row += step_row
            col += step_col

        return target == "--" or target[0] != piece[0]
    
    def is_valid_queen_move(self, piece, start_row, start_col, end_row, end_col, target):
        # Queen moves like both a rook and a bishop
        return (self.is_valid_rook_move(piece, start_row, start_col, end_row, end_col, target) or
                self.is_valid_bishop_move(piece, start_row, start_col, end_row, end_col, target))

    def is_valid_king_move(self, piece, start_row, start_col, end_row, end_col, target):
        # King moves one square in any direction
        if max(abs(start_row - end_row), abs(start_col - end_col)) != 1:
            return False
        return target == "--" or target[0] != piece[0]
    
    def is_castling_move(self, start_row, start_col, end_row, end_col):
        if abs(start_col - end_col) != 2 or start_row != end_row:
            return False

        if (self.current_turn == "white" and start_row != 7) or (self.current_turn == "black" and start_row != 0):
            return False

        direction = 1 if end_col > start_col else -1
        for col in range(start_col + direction, end_col, direction):
            if self.board[start_row][col] != "--":
                return False
            
    def validate_castling_move(self, move):
        start, end = move.split()
        start_col, start_row = ord(start[0]) - ord('a'), 8 - int(start[1])
        end_col, end_row = ord(end[0]) - ord('a'), 8 - int(end[1])

        # Check if the king or rook has moved (you need to track this in your game state)
        if self.has_king_moved(self.current_turn) or self.has_rook_moved_for_castling(start_row, start_col, end_col):
            return False

        # Check if the path between king and rook is clear (already done in is_castling_move)

        # Check if the king is in check, will pass through check, or end in check
        # You need to implement these checks based on your game logic
        if self.is_king_in_check(self.current_turn) or self.will_king_pass_through_check(start_row, start_col, end_col):
            return False

        return True

############# POSSOBLE MOVES

    def get_possible_moves(self):
        possible_moves = []
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece[0] == self.current_turn[0]:  # Check if the piece belongs to the current player
                    moves = self.generate_moves_for_piece(piece, row, col)
                    possible_moves.extend(moves)
        return possible_moves

    def generate_moves_for_piece(self, piece, row, col):
        moves = []
        # Define rules for each type of piece
        if piece[1] == 'p':  # Pawn
            moves = self.generate_pawn_moves(row, col)
        elif piece[1] == 'r':  # Rook
            moves = self.generate_rook_moves(row, col)
        # ... [similar for other pieces: knight, bishop, queen, king]
        return moves

    def generate_pawn_moves(self, row, col):
        # Example for generating pawn moves
        moves = []
        direction = -1 if self.current_turn == 'white' else 1
        # Forward move
        if self.is_empty(row + direction, col):
            moves.append(self.position_to_notation(row, col, row + direction, col))
            # Two-square move from initial position
            if (self.current_turn == 'white' and row == 6) or (self.current_turn == 'black' and row == 1):
                if self.is_empty(row + 2 * direction, col):
                    moves.append(self.position_to_notation(row, col, row + 2 * direction, col))
        # Capture moves
        for capture_col in [col - 1, col + 1]:
            if self.is_capture(row + direction, capture_col):
                moves.append(self.position_to_notation(row, col, row + direction, capture_col))
        return moves

    def generate_rook_moves(self, row, col):
        moves = []
        for d_row, d_col in [(1, 0), (-1, 0), (0, 1), (0, -1)]:  # Directions: up, down, right, left
            for i in range(1, 8):
                new_row, new_col = row + d_row * i, col + d_col * i
                if not self.is_in_bounds(new_row, new_col):
                    break
                if self.board[new_row][new_col] == '--':
                    moves.append(self.position_to_notation(row, col, new_row, new_col))
                elif self.board[new_row][new_col][0] != self.current_turn[0]:
                    moves.append(self.position_to_notation(row, col, new_row, new_col))
                    break
                else:
                    break
        return moves

    def generate_knight_moves(self, row, col):
        moves = []
        for d_row, d_col in [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]:
            new_row, new_col = row + d_row, col + d_col
            if self.is_in_bounds(new_row, new_col) and (self.board[new_row][new_col] == '--' or self.board[new_row][new_col][0] != self.current_turn[0]):
                moves.append(self.position_to_notation(row, col, new_row, new_col))
        return moves

    def generate_bishop_moves(self, row, col):
        moves = []
        for d_row, d_col in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:  # Diagonal directions
            for i in range(1, 8):
                new_row, new_col = row + d_row * i, col + d_col * i
                if not self.is_in_bounds(new_row, new_col):
                    break
                if self.board[new_row][new_col] == '--':
                    moves.append(self.position_to_notation(row, col, new_row, new_col))
                elif self.board[new_row][new_col][0] != self.current_turn[0]:
                    moves.append(self.position_to_notation(row, col, new_row, new_col))
                    break
                else:
                    break
        return moves

    def generate_queen_moves(self, row, col):
        return self.generate_rook_moves(row, col) + self.generate_bishop_moves(row, col)

    def generate_king_moves(self, row, col):
        moves = []
        for d_row in range(-1, 2):
            for d_col in range(-1, 2):
                if d_row == 0 and d_col == 0:
                    continue
                new_row, new_col = row + d_row, col + d_col
                if self.is_in_bounds(new_row, new_col) and (self.board[new_row][new_col] == '--' or self.board[new_row][new_col][0] != self.current_turn[0]):
                    moves.append(self.position_to_notation(row, col, new_row, new_col))
        return moves

    def is_in_bounds(self, row, col):
        return 0 <= row < 8 and 0 <= col < 8

    def generate_moves_for_piece(self, piece, row, col):
        if piece[1] == 'p':
            return self.generate_pawn_moves(row, col)
        elif piece[1] == 'r':
            return self.generate_rook_moves(row, col)
        elif piece[1] == 'n':
            return self.generate_knight_moves(row, col)
        elif piece[1] == 'b':
            return self.generate_bishop_moves(row, col)
        elif piece[1] == 'q':
            return self.generate_queen_moves(row, col)
        elif piece[1] == 'k':
            return self.generate_king_moves(row, col)
        else:
            return []

    def is_empty(self, row, col):
        return 0 <= row < 8 and 0 <= col < 8 and self.board[row][col] == '--'

    def is_capture(self, row, col):
        return 0 <= row < 8 and 0 <= col < 8 and self.board[row][col][0] != self.current_turn[0] and self.board[row][col] != '--'

    def position_to_notation(self, start_row, start_col, end_row, end_col):
        # Converts row/col positions to chess notation (e.g., 'e2 e4')
        return f"{chr(start_col + ord('a'))}{8 - start_row} {chr(end_col + ord('a'))}{8 - end_row}"

    def play_game(self):
        while True:
            self.print_board()
            possible_moves = self.get_possible_moves()
            print(f"{self.current_turn.capitalize()}'s possible moves: {possible_moves}")
            print(f"{self.current_turn}'s turn")
            move = input("Enter your move: ")
            if not self.make_move(move):
                continue

In [6]:
def test_game(output_mode='print'):
    game = ChessGame()
    
    moves = [
        "e2 e4",  # Pawn moves forward
        "e7 e5",  # Opponent's pawn moves forward
        "g1 f3",  # Knight moves
        "b8 c6",  # Opponent's knight moves
        "f1 c4",  # Bishop moves
        "g8 f6",  # Opponent's knight moves
        "e1 g1",  # Attempt to castle king-side
        "e8 g8",  # Opponent attempts to castle king-side
        "e1 c1",  # Attempt to castle queen-side (likely invalid here)
        "e8 c8",  # Opponent attempts to castle queen-side (likely invalid here)
    ]

    def output(text):
        if output_mode == 'file':
            with open("chess_game_output.txt", "a") as file:
                file.write(text + "\n")
        else:
            print(text)

    for move in moves:
        output(f"Attempting move: {move}")
        if game.make_move(move):
            output(f"Move made successfully:")
            board_str = "\n".join([" ".join(row) for row in game.board]) + "\n"
            output(board_str)
            possible_moves = game.get_possible_moves()
            output(f"Possible moves for {game.current_turn}: {', '.join(possible_moves)}")
        else:
            output(f"Invalid move detected")

# To test, you can call test_game with either 'print' or 'file' as the argument.
test_game('print')  # For console output
# test_game('file')  # For file output

Attempting move: e2 e4
Received move: e2 e4
Moved wp from e2 to e4
Turn changed to black
Move made successfully:
br bn bb bq bk bb bn br
bp bp bp bp bp bp bp bp
-- -- -- -- -- -- -- --
-- -- -- -- -- -- -- --
-- -- -- -- wp -- -- --
-- -- -- -- -- -- -- --
wp wp wp wp -- wp wp wp
wr wn wb wq wk wb wn wr

Possible moves for black: b8 c6, b8 a6, g8 h6, g8 f6, a7 a6, a7 a5, b7 b6, b7 b5, c7 c6, c7 c5, d7 d6, d7 d5, e7 e6, e7 e5, f7 f6, f7 f5, g7 g6, g7 g5, h7 h6, h7 h5
Attempting move: e7 e5
Received move: e7 e5
Moved bp from e7 to e5
Turn changed to white
Move made successfully:
br bn bb bq bk bb bn br
bp bp bp bp -- bp bp bp
-- -- -- -- -- -- -- --
-- -- -- -- bp -- -- --
-- -- -- -- wp -- -- --
-- -- -- -- -- -- -- --
wp wp wp wp -- wp wp wp
wr wn wb wq wk wb wn wr

Possible moves for white: a2 a3, a2 a4, b2 b3, b2 b4, c2 c3, c2 c4, d2 d3, d2 d4, f2 f3, f2 f4, g2 g3, g2 g4, h2 h3, h2 h4, b1 c3, b1 a3, d1 e2, d1 f3, d1 g4, d1 h5, e1 e2, f1 e2, f1 d3, f1 c4, f1 b5, f1 a6, g1 h3, g1 f3, g

In [7]:
class ChessGUI:
    def __init__(self, game):
        self.game = game
        self.selected_piece = None
        self.possible_moves = []
        self.root = tk.Tk()
        self.root.title("Chess Game")
        self.canvas = tk.Canvas(self.root, width=800, height=800)
        self.canvas.pack()
        self.draw_board()
        self.canvas.bind("<Button-1>", self.on_square_clicked)

    def draw_board(self):
        self.canvas.delete("all")
        for i in range(8):
            for j in range(8):
                color = "white" if (i + j) % 2 == 0 else "gray"
                self.canvas.create_rectangle(100*j, 100*i, 100*(j+1), 100*(i+1), fill=color)
                self.canvas.create_text(100*j + 5, 100*i + 5, text=f"{chr(j + ord('a'))}{8 - i}", anchor="nw")
                piece = self.game.board[i][j]
                if piece != "--":
                    self.canvas.create_text(100*j + 50, 100*i + 50, text=self.get_piece_symbol(piece), font=("Arial", 50))

        # Highlight possible moves
        for move in self.possible_moves:
            start, end = move.split()
            end_col, end_row = ord(end[0]) - ord('a'), 8 - int(end[1])
            self.canvas.create_rectangle(100*end_col, 100*end_row, 100*(end_col+1), 100*(end_row+1), outline="green", width=4)

    def get_piece_symbol(self, piece):
        # Simple representation of chess pieces using text or emojis
        symbols = {
            "wp": "♙", "bp": "♟",
            "wr": "♖", "br": "♜",
            "wn": "♘", "bn": "♞",
            "wb": "♗", "bb": "♝",
            "wq": "♕", "bq": "♛",
            "wk": "♔", "bk": "♚"
        }
        return symbols.get(piece, "")

    def on_square_clicked(self, event):
        col, row = event.x // 100, event.y // 100
        position = chr(col + ord('a')) + str(8 - row)  # Convert canvas coordinates to chess coordinates (e.g., "e4")
        piece = self.game.board[row][col]

        if self.selected_piece:
            # Attempt to make a move
            move = f"{self.selected_piece} {position}"
            if self.game.make_move(move):
                self.selected_piece = None  # Deselect the piece after a successful move
                self.possible_moves = []  # Clear possible moves
                self.perform_additional_move_actions(move)
                self.draw_board()  # Redraw the board
            else:
                print("Invalid move")
                self.selected_piece = None  # Deselect the piece after an invalid move
                self.possible_moves = []  # Clear possible moves
                self.draw_board()  # Redraw the board
        elif piece != '--' and piece[0] == self.game.current_turn[0]:
            # New piece selected
            self.selected_piece = position
            self.possible_moves = self.get_possible_moves_for_piece(position)
            self.draw_board()

    def perform_additional_move_actions(self, move):
        # Handle special moves like castling
        start, end = move.split()
        start_col, start_row = ord(start[0]) - ord('a'), 8 - int(start[1])
        end_col, end_row = ord(end[0]) - ord('a'), 8 - int(end[1])
        
        if self.game.is_castling_move(start_row, start_col, end_row, end_col):
            # Perform additional actions for castling (such as moving the rook)
            self.handle_castling(start_row, start_col, end_row, end_col)

    def handle_castling(self, start_row, start_col, end_row, end_col):
        # King-side castling
        if end_col == start_col + 2:
            self.game.board[start_row][7] = "--"  # Clear the original rook position
            self.game.board[start_row][start_col + 1] = "wr" if self.game.current_turn == "white" else "br"  # Place the rook next to the king

        # Queen-side castling
        elif end_col == start_col - 2:
            self.game.board[start_row][0] = "--"
            self.game.board[start_row][start_col - 1] = "wr" if self.game.current_turn == "white" else "br"

    def get_possible_moves_for_piece(self, position):
        # Filter possible moves for the selected piece
        all_possible_moves = self.game.get_possible_moves()
        return [move for move in all_possible_moves if move.startswith(position)]

    def start(self):
        self.root.mainloop()


In [8]:
game = ChessGame()
gui = ChessGUI(game)
gui.start()